In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

Import the required libraries.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')
from mediapipe_model_maker import gesture_recognizer
import matplotlib.pyplot as plt

### FSL dataset

> Connect to google drive

In [ ]:
!unzip "/content/drive/MyDrive/THESIS/DATASETALPHA4.zip"
dataset_path = "fsl_alphabet_train_1"

In [ ]:
print(dataset_path)
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

In [ ]:
NUM_EXAMPLES = 4

for label in labels:
  label_dir = os.path.join(dataset_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

## Load the FSL Dataset

In [ ]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

## Train the loaded Dataset

> Based on FSL with 300 pictures every fingerspelling.

In [ ]:
hparams = gesture_recognizer.HParams(
    export_dir="exported_model",
    learning_rate=0.001,
    batch_size=32,
    epochs=50,
    lr_decay=0.95,
    gamma=2,
)
model_options = gesture_recognizer.ModelOptions(
    dropout_rate=0.3,
    layer_widths=[128, 64]
)
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams, model_options=model_options
)
model_2 = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

## Evaluate the model performance


In [ ]:
loss, acc = model_2.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

## Export to Tensorflow Lite Model

In [ ]:
model_2.export_model()
!ls exported_model

In [ ]:
files.download('exported_model/gesture_recognizer.task')